<a href="https://colab.research.google.com/github/JoseLuisPalominoMarmol/Generador_Resumenes/blob/main/generadorResumen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generador de resúmenes


## Instalando librerías

Por defecto, el entorno de Google Colab no tiene instaladas las librerías de [HuggingFace](https://huggingface.co/), por lo que vamos a hacer en primer lugar es instalar las librerías: [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).

In [4]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -q

Después de ejecutar la celda anterior debes reiniciar el entorno desde el menú Runtime -> Restart Session. A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [7]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

Para este ejemplo vamos a utilizar el [dataset de dennlinger/eur-lex-sum](https://huggingface.co/datasets/dennlinger/eur-lex-sum/viewer/spanish) que contiene una serie de noticias y sus respectivos resumenes.

Comenzamos descargando el dataset en español y eliminamosla primera columna que no hará falta, pues corresponde a la identificación con la que enlaza cada noticia.

In [8]:
from datasets import load_dataset, load_metric, Dataset
raw_dataset = load_dataset("dennlinger/eur-lex-sum", "spanish")
raw_dataset = raw_dataset.remove_columns(["celex_id"])
raw_dataset['train']

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for dennlinger/eur-lex-sum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dennlinger/eur-lex-sum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['reference', 'summary'],
    num_rows: 1112
})

Mostramos un ejemplo de cómo luce nuestro dataset.

In [9]:
raw_dataset['train'].to_pandas()

,reference,summary
0,21.6.2019\nES\nDiario Oficial de la Unión Euro...,Folleto que debe publicarse en caso de oferta ...
1,16.5.2019\nES\nDiario Oficial de la Unión Euro...,"Dispositivos de corto alcance, RLAN (Wi-Fi), i..."
2,2.7.2019\nES\nDiario Oficial de la Unión Europ...,Registro de la Unión del régimen de comercio d...
3,28.5.2019\nES\nDiario Oficial de la Unión Euro...,Régimen de comercio de derechos de emisión: no...
4,24.7.2020\nES\nDiario Oficial de la Unión Euro...,Acuerdo UE-China relativo a la seguridad en la...
...,...,...
1107,EUR-Lex - 21986A0618(01) - ES\nAvis juridique ...,Convenio Internacional para la Conservación de...
1108,30.4.2004\nES\nDiario Oficial de la Unión Euro...,Seguro de responsabilidad de las compañías aér...
1109,EUR-Lex - 31965R0019 - ES\nAvis juridique impo...,Normas de la UE sobre prácticas concertadas y ...
1110,EUR-Lex - 31958R0003(01) - ES\nAvis juridique ...,Tratado constitutivo de la Comunidad Europea: ...


Insertamos una librería para trabajar con expresiones regulares. De esta forma podemos limpiar todo el ruido que haya en nuestros textos. En nuestro caso solo nos quedaremos con letras (con y sin acentos) y espacios. Excluiremos los números y los saltos de línea `\n`.

In [10]:
import re
def limpiar_texto(texto):
    patron = r'[^\w\s]'
    texto_limpio = re.sub(patron, '', texto, flags=re.UNICODE)
    texto_limpio = re.sub(r'\d', '', texto_limpio)
    texto_limpio = texto_limpio.replace('\n', ' ')
    return texto_limpio

Pasamos nuestra función por el dataframe y limpiamos ambas columnas de nuestro dataset.  

In [11]:
import pandas as pd
df = raw_dataset['train'].to_pandas()
df['reference'] = df['reference'].apply(limpiar_texto)
df['summary'] = df['summary'].apply(limpiar_texto)
df['combined_clean'] = df['reference'] + " " + df['summary']

In [12]:
df['combined_clean']

0        ES Diario Oficial de la Unión Europea L  REGL...
1        ES Diario Oficial de la Unión Europea L  DECI...
2        ES Diario Oficial de la Unión Europea L  REGL...
3        ES Diario Oficial de la Unión Europea L  REGL...
4        ES Diario Oficial de la Unión Europea L  ACUE...
                              ...                        
1107    EURLex  A  ES Avis juridique important  A Conv...
1108     ES Diario Oficial de la Unión Europea L  REGL...
1109    EURLex  R  ES Avis juridique important  R Regl...
1110    EURLex  R  ES Avis juridique important  R Regl...
1111    EURLex  A  ES Avis juridique important  A Acue...
Name: combined_clean, Length: 1112, dtype: object

In [13]:
#Así ha quedado nuestra columna "reference".
df['reference']

0        ES Diario Oficial de la Unión Europea L  REGL...
1        ES Diario Oficial de la Unión Europea L  DECI...
2        ES Diario Oficial de la Unión Europea L  REGL...
3        ES Diario Oficial de la Unión Europea L  REGL...
4        ES Diario Oficial de la Unión Europea L  ACUE...
                              ...                        
1107    EURLex  A  ES Avis juridique important  A Conv...
1108     ES Diario Oficial de la Unión Europea L  REGL...
1109    EURLex  R  ES Avis juridique important  R Regl...
1110    EURLex  R  ES Avis juridique important  R Regl...
1111    EURLex  A  ES Avis juridique important  A Acue...
Name: reference, Length: 1112, dtype: object

In [14]:
#Así ha quedado nuestra columna "summary".
df['summary']

0       Folleto que debe publicarse en caso de oferta ...
1       Dispositivos de corto alcance RLAN WiFi intern...
2       Registro de la Unión del régimen de comercio d...
3       Régimen de comercio de derechos de emisión nor...
4       Acuerdo UEChina relativo a la seguridad en la ...
                              ...                        
1107    Convenio Internacional para la Conservación de...
1108    Seguro de responsabilidad de las compañías aér...
1109    Normas de la UE sobre prácticas concertadas y ...
1110    Tratado constitutivo de la Comunidad Europea r...
1111    Cooperación tecnológica y científica entre la ...
Name: summary, Length: 1112, dtype: object

Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [dataset de csebuetnlp/mT5_multilingual_XLSum](https://huggingface.co/csebuetnlp/mT5_multilingual_XLSum)

In [25]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=bc928bac5c6e04eb12ae9b368e8df6466182488e57cc67dfbe9082bf4d531246
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorWithPadding

model_checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Definimos una función para tokenizar el texto. En este caso, tokenizaremos ambas columnas.

In [27]:
dataset = Dataset.from_pandas(df)

In [28]:
# Divide el dataset en entrenamiento y evaluación
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [29]:
def tokenize_function(example):
    model_inputs = tokenizer(example['reference'], max_length=256, truncation=True, padding="max_length", return_tensors="pt")

    # Tokenizar el resumen objetivo
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['summary'], max_length=128, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [30]:
# Tokeniza los conjuntos de entrenamiento y evaluación
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [31]:
# Configura el data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [32]:
# Definir la función de evaluación con ROUGE
from datasets import load_metric

def compute_metrics(eval_preds):
    metric = load_metric("rouge")
    logits, labels = eval_preds
    # Decode the generated summaries and references
    decoded_preds = tokenizer.batch_decode(logits, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(pred.split("\n")[:1]) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.split("\n")[:1]) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Extraer las puntuaciones ROUGE específicas
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return result

In [33]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    gradient_accumulation_steps=8,  # Simula un tamaño de lote mayor
    logging_dir='./logs',            # Directorio para almacenar los logs
    logging_strategy='epoch'         # Estrategia de logging para asegurarse de que se muestren las métricas
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [35]:
# Entrena el modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
0,2.849400,2.204007,35.302143,24.733097,29.532309,29.546886
1,2.342200,2.027466,37.301055,25.796402,30.741569,30.736311
2,2.233200,1.988867,37.665842,25.895384,30.796503,30.789519


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

TrainOutput(global_step=165, training_loss=2.4755999940814393, metrics={'train_runtime': 404.0129, 'train_samples_per_second': 6.601, 'train_steps_per_second': 0.408, 'total_flos': 1581541847728128.0, 'train_loss': 2.4755999940814393, 'epoch': 2.966292134831461})

#Compartiendo el modelo
Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [37]:
# Vamos a la carpeta donde se ha guardado nuestro modelo, es el valor que
# definimos previamente en el objeto TrainingArguments
%cd ./results
# Subimos el modelo indicando un mensaje de confirmación, y una etiqueta.
trainer.push_to_hub(commit_message="Training complete", tags="summary")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 84, 'num_beams': 4, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}


/content/results


model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/JoseLuis95/results/commit/e6c9d28820327874966fb9c98a7e91e176ee4e17', commit_message='Training complete', commit_description='', oid='e6c9d28820327874966fb9c98a7e91e176ee4e17', pr_url=None, pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/JoseLuis95/results.
